In [1]:
import numpy as np
import pandas as pd

### point data load(=ldps grid + obs point)

In [40]:
import numpy as np
import netCDF4 as nc
import glob

# nwp grid, stn(bias corrected) load, here example use obs data instead of bias corrected nwp
# nwp grid
#input_dir = "/Users/mankikim/JOB/DATA/"
input_dir = "/Users/mankikim/Desktop/DATA_LINK/"
#files = glob.glob(input_dir + "*.nc")
files = glob.glob(input_dir + "l015_v070_erlo_unis_20kind.2023041000.nc")
# nc_file = nc.Dataset(files[-1])
# print(nc_file)

nwp_grd_u = np.array(nc.Dataset(files[-1]).variables['UGRD_10maboveground'])[0,:,:]
nwp_grd_v = np.array(nc.Dataset(files[-1]).variables['VGRD_10maboveground'])[0,:,:]
print("nwp_u: ", nwp_grd_u.shape)
print("nwp_v: ", nwp_grd_u.shape)
print(nwp_grd_u.min(), nwp_grd_u.max())
print(nwp_grd_v.min(), nwp_grd_v.max())
print(np.where(nwp_grd_v>=20))


nwp_lat = np.load("../fcst_wind/DAIO/nwp/ldaps_lat_grid")
nwp_lon = np.load("../fcst_wind/DAIO/nwp/ldaps_lon_grid")
print("lat shape: ", nwp_lat.shape)
print("lon shape: ", nwp_lon.shape)


# check diff --> 4째자리부터 차이 있음
dif_lon = []
dif_lat = []
#for i in range(nwp_lon.shape[0]):
#    for j in range(nwp_lon.shape[1]-1):
for i in range(100):
    for j in range(100):
        dif_lon.append( np.round(nwp_lon[i,j+1] - nwp_lon[i,j], 5))
        dif_lat.append( np.round(nwp_lat[i+1,j] - nwp_lat[i,j], 5))
dlon = np.array(set(dif_lon))
dlat = np.array(set(dif_lat))
print("dif lon: ", dlon)
print("dif lat: ", dlat)


# # LDAPS 9999값 확인 필요 
# idx = np.where( nwp_grd_u < 999. )
#index = set(np.where(np.isnan(nwp_data))[0])
np.where( nwp_grd_u < 999. )
#print(a.shape)
#sub_nwp_grd_u = nwp_grd_u[nwp_grd_u < 999.]
#print(sub_nwp_grd_u.shape)
#for i in range(nwp_lat.shape[0]):
#    for j in range(nwp_lat.shape[1]):



nwp_u:  (781, 602)
nwp_v:  (781, 602)
-14.243284 9.999e+20
-8.791781 9.999e+20
(array([  0,   0,   0, ..., 780, 780, 780], dtype=int64), array([593, 594, 595, ..., 599, 600, 601], dtype=int64))
lat shape:  (781, 602)
lon shape:  (781, 602)
dif lon:  {0.01632, 0.01643, 0.01615, 0.01626, 0.01637, 0.01609, 0.0162, 0.01631, 0.01642, 0.01614, 0.01625, 0.01636, 0.01608, 0.01619, 0.0163, 0.01613, 0.01641, 0.01624, 0.01635, 0.01618, 0.01629, 0.01612, 0.0164, 0.01623, 0.01634, 0.01617, 0.01628, 0.01639, 0.01611, 0.01622, 0.0161, 0.01633, 0.01616, 0.01627, 0.01638, 0.01621}
dif lat:  {0.01364, 0.01361, 0.01365, 0.01362, 0.01366, 0.01367, 0.01363, 0.0136}


(array([  0,   0,   0, ..., 780, 780, 780], dtype=int64),
 array([  0,   1,   2, ..., 590, 591, 592], dtype=int64))

In [34]:
import numpy as np

# Create a NumPy array
array = np.array([[True, False, True],
                  [False, True, False]])
print(array.shape)
# Get the true indices
true_indices = np.argwhere(array)

# Print the true indices
print(true_indices)

(2, 3)
[[0 0]
 [0 2]
 [1 1]]


In [ ]:
# obs stn for 불규칙 데이터
obs_stnd = np.load("../fcst_wind/DAIO/obs_data_47105")
print(obs_stnd.shape)


# select U & V
#if sel_v == 'u':
#    nwp_grid[:,:,0]

#### 영역분리

In [ ]:
# # 37~39, 127~129
# print(nwp_lat.shape)
# print(nwp_lon.shape)
# #lat_mask = np.where(nwp_lat>=37.0 and nwp_lat<=39.0)
# #ll_mask = np.where(np.logical_and(nwp_lat>=37.0, nwp_lat<=39.0, nwp_lon>=127.0, nwp_lon<=129.0) )
# idx = (nwp_lat >= 37.0) & (nwp_lat <= 39.0) & (nwp_lon >= 127.0) & (nwp_lon <= 129.0)

# print(idx)
# sub_nwp_lat = nwp_lat[(nwp_lat >= 37.0) & (nwp_lat <= 39.0) & (nwp_lon >= 127.0) & (nwp_lon <= 129.0)]

# print(sub_nwp_lat.shape)
# #sub_nwp_grd_u = nwp_grd_u[ll_mask]
# #sub_nwp_grd_v = nwp_grd_v[ll_mask]


### 불규칙데이터 생성

In [15]:
# 인풋 shape 만들기
flatten_input = []
for i in range(nwp_lat.shape[0]): # lat
    for j in range(nwp_lat.shape[1]): # lon
        flatten_input.append([ nwp_lat[i,j], nwp_lon[i,j], nwp_grd_u[0,i,j] ])


# asarray(copy=False)
print("ldps points: ",len(flatten_input))
input_data = np.asarray(flatten_input)
print(input_data[0:5])

# extract attribute 
lon_lat_data = input_data[:, 0:2]
qff_values = input_data[:, 2]

print(lon_lat_data[0][0], ',', lon_lat_data[0][1])
print(lon_lat_data[-1][0], ',', lon_lat_data[-1][1])
print(qff_values[0])

print(lon_lat_data.shape)
print(qff_values.shape)
print(np.min(qff_values), np.max(qff_values))




ldps points:  470162
[[ 32.256875   121.834429    -0.7100811 ]
 [ 32.25758153 121.85050501  -0.58459282]
 [ 32.25828532 121.86658136  -0.46105766]
 [ 32.2589864  121.88265804  -0.40197563]
 [ 32.25968474 121.89873506  -0.41564751]]
32.256875000000015 , 121.83442899999999
42.93468114328637 , 132.531874170454
-0.7100811004638672
(470162, 2)
(470162,)
-14.243284225463867 9.999000260554009e+20


In [ ]:
#!C:\Users\tjoeun\AppData\Local\Programs\Python\Python39\Scripts\pip install fast-barnes-py

### Objective analysis - barnes

In [13]:
#!pip install fast-barnes-py
# definition of a 12Â° x 12Â° grid starting at 9Â°W / 47Â°N
# resolution = 200.
# step = 1.0 / resolution # 1도안에 넣을개수
# x0 = np.asarray([lon_lat_data[0][0], lon_lat_data[0][1]], dtype=np.float64)
# size = (int(2.0 / step), int(2.0 / step)) # 37~39, 127~129
# Xend1 = x0[0] + (step*size[0])
# Xend2 = x0[1] + (step*size[1])


lat_dist = abs(lon_lat_data[:,0].min() - lon_lat_data[:,0].max())
lon_dist = abs(lon_lat_data[:,1].min() - lon_lat_data[:,1].max())
step = 0.005
x0 = np.asarray([lon_lat_data[0][0], lon_lat_data[0][1]], dtype=np.float64)
size = (int(lat_dist / step), int(lon_dist / step))
x_e_lat = x0[0] + (step*size[0])
x_e_lon = x0[1] + (step*size[1])
print(x0)
print(step)
print(size)
print(x_e_lat, x_e_lon)



# calculate Barnes interpolation
from fastbarnes import interpolation
sigma = 1.0
field = interpolation.barnes(lon_lat_data, qff_values, sigma, x0, step, size)
#print(type(field))
#print(np.round(field, 2))

field.shape
field_1 = np.where( (field<=20.) & (field>=-20.) , field, 0.)

[ 32.256875 121.834429]
0.005
(2188, 2294)
43.19687500000001 133.304429


In [14]:
print(field_1.min())
print(field_1.max())

0.0
0.0


In [ ]:
# draw graphic with labeled contours and scattered sample points
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 5))

gridX = np.arange(x0[0], x0[0]+size[1]*step, step)
gridY = np.arange(x0[1], x0[1]+size[0]*step, step)
levels = np.arange(-20, 20, 1)
cs = plt.contour(gridX, gridY, field, levels)
plt.clabel(cs, levels[::2], fmt='%d', fontsize=9)

#plt.scatter(lon_lat_data[:, 0], lon_lat_data[:, 1], color='red', s=20, marker='.')

plt.show()

In [ ]:
# input_data = np.asarray([
#     [-3.73,56.33,995.1], [2.64,47.05,1012.5], [-8.40,47.50,1011.3], [2.94,54.33,1006.0], [-2.90,49.90,1006.3],
#     [-8.98,53.72,1002.1], [1.20,58.60,1002.6], [1.60,50.73,1009.1], [-7.38,57.36,997.7], [-1.25,53.01,1000.4],
#     [-4.74,52.79,998.4], [-0.61,47.48,1013.0], [-6.10,50.10,1004.3], [-6.46,54.87,996.4], [-3.22,47.29,1012.8],
#     [-1.60,55.42,996.6], [2.30,56.60,1004.5], [1.12,52.95,1003.6], [-0.90,57.80,999.9], [-7.90,51.40,1002.6],
#     [-0.70,50.10,1007.5], [2.53,49.02,1010.8], [-5.06,48.47,1008.5], [-3.10,53.60,997.5], [-5.63,57.86,997.8],
#     [-6.90,52.85,1000.9], [-4.15,51.09,1002.6], [-1.99,51.50,1002.7], [1.21,47.68,1011.7], [-5.70,56.30,995.5],
#     [-1.98,53.13,998.5], [1.09,49.93,1009.0], [1.72,58.42,1002.9], [-6.30,52.30,999.4], [0.70,57.70,1001.9],
#     [-3.50,53.60,995.9], [1.38,48.06,1011.6], [-4.37,51.71,1001.1], [-3.09,58.45,998.5], [2.00,56.40,1003.9],
#     [1.90,57.00,1003.3], [0.45,51.90,1004.9], [-8.25,51.80,1002.5], [-1.87,53.81,997.4], [-2.38,55.71,995.1],
#     [-4.01,54.80,992.1], [0.88,53.37,1002.6], [-1.69,51.86,1002.1], [-4.57,52.14,999.6], [-0.20,58.40,1001.1],
# ])

# lon_lat_data = input_data[:, 0:2]
# qff_values = input_data[:, 2]
# input_data.shape

# print(input_data[:,0].min())
# print(input_data[:,0].max())
# print(input_data[:,0].min() - input_data[:,0].max())
# print(lon_lat_data[:,0])

# # definition of a 12Â° x 12Â° grid starting at 9Â°W / 47Â°N
# resolution = 32.0
# step = 1.0 / resolution
# x0 = np.asarray([-9.0, 47.0], dtype=np.float64)
# size = (int(12.0 / step), int(12.0 / step))
# Xend1 = x0[0] + (step*size[0])
# Xend2 = x0[1] + (step*size[1])

# print(x0)
# print(step)
# print(size)
# print(Xend1, Xend2)

# # calculate Barnes interpolation
# from fastbarnes import interpolation
# sigma = 1.0
# field = interpolation.barnes(lon_lat_data, qff_values, sigma, x0, step, size)


# # draw graphic with labeled contours and scattered sample points
# import matplotlib.pyplot as plt
# plt.figure(figsize=(5, 5))

# gridX = np.arange(x0[0], x0[0]+size[1]*step, step)
# gridY = np.arange(x0[1], x0[1]+size[0]*step, step)
# levels = np.arange(976, 1026, 2)
# cs = plt.contour(gridX, gridY, field, levels)
# plt.clabel(cs, levels[::2], fmt='%d', fontsize=9)

# plt.scatter(lon_lat_data[:, 0], lon_lat_data[:, 1], color='red', s=20, marker='.')

# plt.show()